<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Solution Homework 1,</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

# Solution Homework 1
In this notebook, we propose a possible solution for the first exercise proposed during the lectures.

### Software Requirements
Be sure of running the following snippet in order to install in the notebook all the software needed to execute this notebook.

In [0]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

## Solution Script
We start the model definition by declaring the input data of our problem.
Since it is a toy problem, we can use just a few *pairs* to store the data. Ideally, the input data should be stored in a CSV, Excel, or JSON file.

In [0]:
# Input Data
rB, rC = 200, 140    # Production rate tons/hour
pB, pC = 25, 30      # Profits euro/tons
dB, dC = 6000, 4000  # Max demands in tons
T = 40               # Maximum working hours per week (for both)

First, we define an empty **ConcreteModel** object, which will contain all the elements of our model.

In [0]:
from pyomo.environ import *
model = ConcreteModel()

For using the object **Variable**, you can look at the example in the [official documentation](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Variables.html)

In [0]:
# declare decision variables
model.xB = Var(domain=NonNegativeReals, bounds= (0.0, dB), initialize=0.0)
model.xC = Var(domain=NonNegativeReals, bounds= (0.0, dC), initialize=0.0)

At this point, we define the **Objective** function, using the variables just defined.

In [0]:
# declare objective
model.cost = Objective(
    expr = pB * model.xB + pC * model.xC,
    sense = maximize)

Finally, we add the only **Constraint** left, on the total number of weekly working hours.

In [0]:
# declare constraints
model.cnstr1 = Constraint(expr = model.xB/rB + model.xC/rC <= T)

In order to solve the model, we need to construct the linear model, by bidding the formal parameters to the actual values, and, only then, we can invoke an **LP solver** to actually solve the model. By using the option `tee = True`, we get in a cell all the solver output:

In [0]:
# Using the GLPK open source MILP solver:
sol = SolverFactory('glpk').solve(model, tee=True)

Before getting the solution, we should check if the solver was able to compute any feasible solution. 

**REMARK:** An LP problem can be:

1. **Feasible**: Hence, a finite optimal solution exists.
2. **Unfeasible**: the feasible region defined by the linear constraints is empty.
3. **Unbounded**: there exists an unbounded direction of grow for the variables, and not finite optimal solution exists.

For this reason, it is good practice [to query the solver for its status](https://pyomo.readthedocs.io/en/stable/working_models.html#accessing-solver-status) before querying the model for the actual solution values.

In [0]:
print(sol.solver.status)

At this point, if we need to check the optimal slution values found by the solver, we can "invoke" the variables and the objective function, as follows.

In [0]:
print("x1 = {}, x2 = {}".format(model.xB(), model.xC()))

In [0]:
print("f(x1,x2) = {}", model.cost())

## Main Pyomo Classes
You should study the following Pyomo documentation for the following three classes:

1. [Var](https://pyomo.readthedocs.io/en/stable/library_reference/aml/index.html#pyomo.environ.Var)
2. [Objective](https://pyomo.readthedocs.io/en/stable/library_reference/aml/index.html#pyomo.environ.Objective)
3. [Constraint](https://pyomo.readthedocs.io/en/stable/library_reference/aml/index.html#pyomo.environ.Constraint)

The other main elements are described in the [Pyomo Algebraic Modeling Lanaguge](https://pyomo.readthedocs.io/en/stable/library_reference/aml/index.html) documentation.


## Graphical Solution
For problems with just two variables, we can always solve the problem graphically, and to get some intuition of what will happen when the decision variables are vectors of $\mathbb{R}^n$.

In the following snippets, we first plot the half-space delimited by the bound constraints, and the production time constraints.

Then, we plot the gradients and a line parallel to the objective function.

Finally, we plot for two corners of the polyhedron associated to the feasible set of points, the directions perpedincular to the active constraints: note that in the optimal solution $x^* = (6000, 1400)$, the gradient of the objective function is contained in the cone define by the two directions perpendicular to the active constraints.

In [0]:
from pylab import *
figure(figsize=(6, 6))
subplot(111, aspect='equal')
M = 8000 # Max bound for plotting
axis([0, M, 0, M])
xlabel('$x_B$')
ylabel('$x_C$')

# PLOT OF ACTIVE CONSTRAINTS
#----------------------------
# Demand for bands
plot([dB,dB],[0, M],'b',lw=2)
fill_between([dB, M], [M, M], color='b', alpha=0.25)
# Demand for coils
plot([0,M],[dC,dC],'c',lw=2)
fill_between([0, M], [dC, dC], [M, M], color='c', alpha=0.25)
# First constraint
x = array([0, M])
y = rC*(T - x/rB)
plot(x, y, 'g', lw=2)
fill_between([0, M], [5600, 0], [M, M], color='g', alpha=0.25)

# OBJECTIVE FUNCTION
#--------------------
annotate('', xy=(2500/2, 3000/2), xytext=(0, 0),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='r'))

x = array([0, M])
for p in linspace(0, 160000.0, 20):
    y = 1/pC*(p-pB*x)
    plot(x, y, 'y--', color='red', alpha=0.3)

# DIRECTION OF ACTIVE CONSTRAINTS AND GRADIENTS
#-----------------------------------------------
# Cone for the optimal solution x* = (6000, 1400)
plot(model.xB(), model.xC(), 'r.', ms=20)
annotate('', xy=(model.xB()+2500/2, model.xC()+3000/2), xytext=(model.xB(), model.xC()),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='r'))

annotate('', xy=(model.xB()+700, model.xC()+1000), xytext=(model.xB(), model.xC()),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='g'))

annotate('', xy=(model.xB()+1000, model.xC()), xytext=(model.xB(), model.xC()),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='b'))

# Cone for the second vertex
plot(rB*(40 - 4000/rC), 4000, 'r.', ms=20)
annotate('', xy=(rB*(40 - 4000/rC)+2500/2, 4000+3000/2), xytext=(rB*(40 - 4000/rC), 4000),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='r'))

annotate('', xy=(rB*(40 - 4000/rC)+700, 4000+1000), xytext=(rB*(40 - 4000/rC), 4000),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='g'))

annotate('', xy=(rB*(40 - 4000/rC), 4000+1000), xytext=(rB*(40 - 4000/rC), 4000),
          arrowprops=dict(shrink=.1, width=2, headwidth=10, color='c'))